<a href="https://colab.research.google.com/github/SytzeAndr/NGCF_RP32/blob/master/NGCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import random as rd

**Data class**

The Data class is a utility class that will help loading the data, compute useful properties of the data and create sample batches.

In [2]:
class Data(object):
  def __init__(self, path, batch_size):
    self.path = path
    self.batch_size = batch_size

    train_file = path + '/train.txt'
    test_file = path + '/test.txt'

    self.n_train = 0
    self.n_test = 0
    self.exist_users = []
    self.train_items = {}
    self.test_items = {}

    with open(train_file) as f:
      for l in f.readlines():
        if len(l) > 0:
          l = l.strip('\n')
          items = [int(i) for i in l.split(' ')]
          uid, train_items = items[0], items[1:]
          self.exist_users.append(uid)
          self.train_items[uid] = train_items
          self.n_train += len(train_items)

    with open(test_file) as f:
      for l in f.readlines():
        if len(l) > 0:
          l = l.strip('\n')
          try:
            items = [int(i) for i in l.split(' ')]
            uid, test_items = items[0], items[1:]
            self.exist_users.append(uid)
            self.test_items[uid] = test_items
            self.n_test += len(test_items)
          except Exception:
            continue

    train_max_item = max([max(items) for items in list(self.train_items.values())])
    test_max_item = max([max(items) for items in list(self.test_items.values())])
    self.n_items = max(train_max_item, test_max_item)
    self.n_users = max(self.exist_users)
  

  def sample(self):
    if self.batch_size <= self.n_users:
      users = rd.sample(self.exist_users, self.batch_size)
    else:
      users = [rd.choice(self.exist_users) for _ in range(self.batch_size)]


    def sample_pos_items_for_u(u, num):
      pos_items = self.train_items[u]
      n_pos_items = len(pos_items)
      pos_batch = []
      while True:
        if len(pos_batch) == num: break
        pos_id = np.random.randint(low=0, high=n_pos_items, size=1)[0]
        pos_i_id = pos_items[pos_id]

        if pos_i_id not in pos_batch:
          pos_batch.append(pos_i_id)
      return pos_batch


    def sample_neg_items_for_u(u, num):
      neg_batch = []
      while True:
        if len(neg_batch) == num: break
        neg_id = np.random.randint(low=0, high=self.n_items, size=1)[0]
        if neg_id not in self.train_items[u] and neg_id not in neg_batch:
          neg_batch.append(neg_id)
      return neg_batch
    

    pos_items, neg_items = [], []
    for u in users:
      pos_items += sample_pos_items_for_u(u, 1)
      neg_items += sample_neg_items_for_u(u, 1)

    return users, pos_items, neg_items

data = Data('.', 10)

FileNotFoundError: ignored

In [0]:
data.sample()